# QNN on Pynq

This notebook covers how to use low quantized Neural Networks on Pynq for inference on MNIST dataset by using LFC network composed of 4 fully connected layers with 1024 neurons each. There are 2 networks using different precision: 

- LFCW1A1 using 1 bit weights and 1 activation,
- LFCW1A2 using 1 bit weights and 2 activation

In [ ]:
import bnn
import os
from copy import deepcopy
from json import dumps
from yapf.yapflib.yapf_api import FormatCode
import matplotlib.pyplot as plt
from pynq import Xlnk

## 1. LFC and MNIST

This notebook performs inference on MNIST test set from http://yann.lecun.com/exdb/mnist/ which contains 10000 pictures of handwritten digits. The LFC network requires MNIST formatted input data, that's why the binary test file can be directly loaded. All other images have to be formatted to this specification (refer to url and LFC webcam examples).

At first you need to download mnist test set and labels using wget and unzip the archive as shown below:
In order to be able to compare the inferred classes against the expected labels we first read the labels:

In [ ]:
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/t10k-images-idx3-ubyte.gz"):
    #get
    !wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz 
    !wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz 
        
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/t10k-images-idx3-ubyte"):
    #unzip    
    !gzip -d t10k-images-idx3-ubyte.gz
    !gzip -d t10k-labels-idx1-ubyte.gz

#read labels
labels = []
with open("/home/xilinx/jupyter_notebooks/bnn/t10k-labels-idx1-ubyte","rb") as lbl_file:
    #read magic number and number of labels (MSB first) -> MNIST header
    magicNum = int.from_bytes(lbl_file.read(4), byteorder="big")
    countLbl = int.from_bytes(lbl_file.read(4), byteorder="big")
    #now the labels are following byte-wise
    for idx in range(countLbl):
        labels.append(int.from_bytes(lbl_file.read(1), byteorder="big"))
    lbl_file.close()
len(labels)

## 2. Hardware Inference

First of all a classifier needs to be instantiated. Using the LfcClassifier will allow to classify MNIST formatted images utilizing LFC network. There are two different runtimes available: hardware accelerated and pure software environment.

Once a classifier is instantiated the inference on MNIST images can be started using `classify_mnist` or `classify_mnists` methods - for both single and multiple images.

In [ ]:
xlnk = Xlnk()

num_runs = 200
input_file = "/home/xilinx/jupyter_notebooks/bnn/t10k-images-idx3-ubyte"
output_folder = "/home/xilinx/jupyter_notebooks/bnn/{}flips/lfc/"

### Utility Functions

In [ ]:
def calculate_accuracy(results, labels):
    countRight = 0
    for idx in range(len(labels)):
        if labels[idx] == results[idx]:
            countRight += 1
    return countRight*100/len(labels)


#Merge dictionaries that contain sub-dictionaries
def dict_of_dicts_merge(*dicts):
    out = {}
    for item in dicts:
        overlapping_keys = out.keys() & item.keys()
        for key in overlapping_keys:
            if (isinstance(out[key], dict) and isinstance(item[key], dict)):
                out[key] = dict_of_dicts_merge(out[key], item[key])
        for key in item.keys() - overlapping_keys:
            out[key] = deepcopy(item[key])
    return out


#Make a dictionary for storing the results, times, or accuracies from a test
def make_results_dict(size, targeting_bits_and_words, targeting_weights_and_activations):
    if targeting_bits_and_words and targeting_weights_and_activations:
        return {
            "bit": {
                "weight": [ None for i in range(size) ],
                "activation": [ None for i in range(size) ],
            },
            "word": {
                "weight": [ None for i in range(size) ],
                "activation": [ None for i in range(numsize_runs) ],
            },
        }
    elif targeting_bits_and_words:
        return {
            "bit": [ None for i in range(size) ],
            "word": [ None for i in range(size) ],
        }
    elif targeting_weights_and_activations:
        return {
            "weight": [ None for i in range(size) ],
            "activation": [ None for i in range(numsize_runs) ],
        }


#Output dict contains the combined raw results from a test. Used as input to calculate_stats
def make_output_dict(network_name, num_runs, num_flips, accuracy_control, accuracies_dict):
    out = {}
    out["network"] = network_name
    out["run count"] = num_runs
    out["flips"] = num_flips
    out["control"] = accuracy_control
    out["results"] = {}
    for key, val in accuracies_dict.items():
        out["results"][key] = val
    return out


#Calculates various stats given the results of a test (formatted with make_output_dict)
def calculate_stats(output_dict):
    out = output_dict.copy()
    for key, value in output_dict["results"].items():
        out["results"][key] = {}
        out["results"][key]["runs"] = {}
        out["results"][key]["runs"]["all"] = value
        out["results"][key]["runs"]["effective"] = list(filter(lambda x: x != out["control"], value))
        out["results"][key]["effective count"] = len(out["results"][key]["runs"]["effective"])
        if out["results"][key]["effective count"] != 0:
            out["results"][key]["avg accuracy"] = sum(value) / len(value)
            out["results"][key]["avg effective accuracy"] = sum(out["results"][key]["runs"]["effective"]) / out["results"][key]["effective count"]
            out["results"][key]["effective accuracy delta"] = out["control"] - out["results"][key]["avg effective accuracy"]
        out["results"][key]["min accuracy"] = min(value)
        out["results"][key]["max accuracy"] = max(value)
    return out


def dict_to_str(dict):
    dict_string = dumps(dict)
    formatted_code, _ = FormatCode(dict_string)
    return formatted_code


def write_stats_file(file_name, stats_dict):
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    f = open(file_name, "w+")
    f.write(dict_to_str(stats_dict))
    f.close()

### Fault Test Function

In [ ]:
def lfc_mnist_fault_test(title, network, input_file, num_runs, num_flips, flip_word, weight_or_activation, target_layers = []):
    results    = [ None for i in range(num_runs) ]
    times      = [ None for i in range(num_runs) ]
    accuracies = [ None for i in range(num_runs) ]

    for i in range(num_runs):
        hw_classifier = bnn.LfcClassifier(network, 'mnist', bnn.RUNTIME_HW)
        print("{} run {} of {} (flipping {} {}(s) at random times)".format(title, i+1, num_runs, num_flips, "word" if flip_word else "bit"))

        results[i]    = hw_classifier.classify_mnists_with_faults(input_file, num_flips, flip_word, weight_or_activation, target_layers)
        times[i]      = hw_classifier.usecPerImage
        accuracies[i] = calculate_accuracy(results[i], labels)

        print("Accuracy:", accuracies[i])

        xlnk.xlnk_reset()
        print()
    
    return (results, times, accuracies)

### W1A1 - 1 bit weights and 1 bit activation

In [ ]:
#Control run
accuracy_W1A1_control = 0
result_W1A1_control = None
time_W1A1_control = 0

#Control test
hw_classifier = bnn.LfcClassifier(bnn.NETWORK_LFCW1A1, "mnist", bnn.RUNTIME_HW)

print("Control classification")
result_W1A1_control   = hw_classifier.classify_mnists(input_file)
time_W1A1_control     = hw_classifier.usecPerImage
accuracy_W1A1_control = calculate_accuracy(result_W1A1_control, labels)
print("Accuracy:", accuracy_W1A1_control)

#Reset the device
xlnk.xlnk_reset()
print()

In [ ]:
def test_W1A1(num_runs, num_flips, output_folder):
    weight_bit_output = {}
    activation_bit_output = {}
    weight_word_output = {}
    activation_word_output = {}
    stats = {}

    #Test bit flips (weight)
    weight_bit_data   = lfc_mnist_fault_test("Weight Bit", bnn.NETWORK_LFCW1A1, input_file, num_runs, num_flips, False, 0)
    weight_bit_output = make_output_dict("lfcW1A1", num_runs, num_flips, accuracy_W1A1_control, {"weight bit": weight_bit_data[2]})

    write_stats_file(output_folder.format(num_flips) + "temp/lfcW1A1_results_bit_weight.txt", weight_bit_output)


    #Test bit flips (activation)
    activation_bit_data   = lfc_mnist_fault_test("Activation Bit", bnn.NETWORK_LFCW1A1, input_file, num_runs, num_flips, False, 1)
    activation_bit_output = make_output_dict("lfcW1A1", num_runs, num_flips, accuracy_W1A1_control, {"activation bit": activation_bit_data[2]})

    write_stats_file(output_folder.format(num_flips) + "temp/lfcW1A1_results_bit_activation.txt", activation_bit_output)


    #Test word flips (weight)
    weight_word_data   = lfc_mnist_fault_test("Weight Word", bnn.NETWORK_LFCW1A1, input_file, num_runs, num_flips, True, 0)
    weight_word_output = make_output_dict("lfcW1A1", num_runs, num_flips, accuracy_W1A1_control, {"weight word": weight_word_data[2]})

    write_stats_file(output_folder.format(num_flips) + "temp/lfcW1A1_results_word_weight.txt", weight_word_output)


    #Test word flips (activation)
    activation_word_data   = lfc_mnist_fault_test("Activation Word", bnn.NETWORK_LFCW1A1, input_file, num_runs, num_flips, True, 1)
    activation_word_output = make_output_dict("lfcW1A1", num_runs, num_flips, accuracy_W1A1_control, {"activation word": activation_word_data[2]})

    write_stats_file(output_folder.format(num_flips) + "temp/lfcW1A1_results_word_activation.txt", activation_word_output)


    #Write all stats to file
    stats = dict_of_dicts_merge(weight_bit_output, activation_bit_output, weight_word_output, activation_word_output)
    stats = calculate_stats(stats)
    write_stats_file(output_folder.format(num_flips) + "lfcW1A1_stats.txt", stats)

### W1A2 - 1 bit weights and 2 bit activation

In [ ]:
#Control run
accuracy_W1A2_control = 0
result_W1A2_control = None
time_W1A2_control = 0

#Control test
hw_classifier = bnn.LfcClassifier(bnn.NETWORK_LFCW1A2, "mnist", bnn.RUNTIME_HW)

print("Control classification")
result_W1A2_control  = hw_classifier.classify_mnists(input_file)
time_W1A2_control     = hw_classifier.usecPerImage
accuracy_W1A2_control = calculate_accuracy(result_W1A2_control, labels)
print("Accuracy:", accuracy_W1A1_control)

#Reset the device
xlnk.xlnk_reset()
print()

In [ ]:
def test_W1A2(num_runs, num_flips, output_folder):
    weight_bit_output = {}
    activation_bit_output = {}
    weight_word_output = {}
    activation_word_output = {}
    stats = {}

    #Test bit flips (weight)
    weight_bit_data   = lfc_mnist_fault_test("Weight Bit", bnn.NETWORK_LFCW1A2, input_file, num_runs, num_flips, False, 0)
    weight_bit_output = make_output_dict("lfcW1A2", num_runs, num_flips, accuracy_W1A2_control, {"weight bit": weight_bit_data[2]})

    write_stats_file(output_folder.format(num_flips) + "temp/lfcW1A2_results_bit_weight.txt", weight_bit_output)


    #Test bit flips (activation)
    activation_bit_data   = lfc_mnist_fault_test("Activation Bit", bnn.NETWORK_LFCW1A2, input_file, num_runs, num_flips, False, 1)
    activation_bit_output = make_output_dict("lfcW1A2", num_runs, num_flips, accuracy_W1A2_control, {"activation bit": activation_bit_data[2]})

    write_stats_file(output_folder.format(num_flips) + "temp/lfcW1A2_results_bit_activation.txt", activation_bit_output)


    #Test word flips (weight)
    weight_word_data   = lfc_mnist_fault_test("Weight Word", bnn.NETWORK_LFCW1A2, input_file, num_runs, num_flips, True, 0)
    weight_word_output = make_output_dict("lfcW1A2", num_runs, num_flips, accuracy_W1A2_control, {"weight word": weight_word_data[2]})

    write_stats_file(output_folder.format(num_flips) + "temp/lfcW1A2_results_word_weight.txt", weight_word_output)


    #Test word flips (activation)
    activation_word_data   = lfc_mnist_fault_test("Activation Word", bnn.NETWORK_LFCW1A2, input_file, num_runs, num_flips, True, 1)
    activation_word_output = make_output_dict("lfcW1A2", num_runs, num_flips, accuracy_W1A2_control, {"activation word": activation_word_data[2]})

    write_stats_file(output_folder.format(num_flips) + "temp/lfcW1A2_results_word_activation.txt", activation_word_output)


    #Write all stats to file
    stats = dict_of_dicts_merge(weight_bit_output, activation_bit_output, weight_word_output, activation_word_output)
    stats = calculate_stats(stats)
    write_stats_file(output_folder.format(num_flips) + "lfcW1A2_stats.txt", stats)

## 3. Run Tests

In [ ]:
test_W1A1(num_runs, 1, output_folder)
test_W1A2(num_runs, 1, output_folder)

test_W1A1(num_runs, 2, output_folder)
test_W1A2(num_runs, 2, output_folder)

test_W1A1(num_runs, 5, output_folder)
test_W1A2(num_runs, 5, output_folder)

test_W1A1(num_runs, 10, output_folder)
test_W1A2(num_runs, 10, output_folder)

test_W1A1(num_runs, 20, output_folder)
test_W1A2(num_runs, 20, output_folder)

test_W1A1(num_runs, 50, output_folder)
test_W1A2(num_runs, 50, output_folder)

test_W1A1(num_runs, 100, output_folder)
test_W1A2(num_runs, 100, output_folder)